Reference :

https://www.kaggle.com/yasufuminakama/moa-pytorch-nn-starter

In [1]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
import random

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import log_loss

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/test_features.csv
/kaggle/input/iterative-stratification/iterative-stratification-master/.gitignore
/kaggle/input/iterative-stratification/iterative-stratification-master/LICENSE
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.cfg
/kaggle/input/iterative-stratification/iterative-stratification-master/.travis.yml
/kaggle/input/iterative-stratification/iterative-stratification-master/README.md
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.py
/kaggle/input/iterative-stratification/iterative-stratification-master/tests/test_ml_stratifiers.py
/kaggle/input/iterative-stratification/iterative-stratification-master/tests/__init__.py
/kaggle/input/iterative-stratification/iterative-stratification-master/iterstrat/ml_stratifiers.py

In [3]:
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/__notebook__.ipynb


In [4]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F

In [5]:
train_features=pd.read_csv('../input/lish-moa/train_features.csv')
test_features=pd.read_csv('../input/lish-moa/test_features.csv')
train_targets_scored=pd.read_csv('../input/lish-moa/train_targets_scored.csv')
sample_submission=pd.read_csv('../input/lish-moa/sample_submission.csv')

In [6]:
tr_ctrl_vehicle_idx=train_features[train_features['cp_type']=='ctl_vehicle']['sig_id'].values
te_ctrl_vehicle_idx=test_features[test_features['cp_type']=='ctl_vehicle']['sig_id'].values


In [7]:
train_features=train_features[~train_features['sig_id'].isin(tr_ctrl_vehicle_idx)]
test_features=test_features[~test_features['sig_id'].isin(te_ctrl_vehicle_idx)]
train_targets_scored=train_targets_scored[~train_targets_scored['sig_id'].isin(tr_ctrl_vehicle_idx)]

In [8]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
train_features.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [10]:
##scaling the features,
for col in train_features.columns[4:]:
    mean,std=train_features[col].mean(),train_features[col].std()
    print(f'Standardizing {col} with mean {mean} and std {std}')
    train_features[col]=(train_features[col]-mean)/std
    test_features[col]=(test_features[col]-mean)/std

Standardizing g-0 with mean 0.27767496810643344 and std 1.4271207461708488
Standardizing g-1 with mean -0.07984522963368006 and std 0.8019541944616344
Standardizing g-2 with mean 0.16839111536358686 and std 1.0357960655603637
Standardizing g-3 with mean 0.08126479861490798 and std 0.9551166268562821
Standardizing g-4 with mean 0.08497379715691607 and std 1.023297028140235
Standardizing g-5 with mean -0.14338197102241745 and std 1.183272373245025
Standardizing g-6 with mean 0.012125505740841893 and std 0.8710035960674436
Standardizing g-7 with mean -0.20433155640605224 and std 1.1368235963422506
Standardizing g-8 with mean -0.23986487151448976 and std 1.8001942558970068
Standardizing g-9 with mean 0.10428110989611707 and std 1.1014531104483078
Standardizing g-10 with mean -0.13397369691999195 and std 1.3136343644444792
Standardizing g-11 with mean 0.2028165071988361 and std 1.2778977148493642
Standardizing g-12 with mean 0.1076905139420456 and std 1.2315789848943517
Standardizing g-13 w

In [11]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [12]:
seed_everything(seed=42)

In [13]:
targets=train_targets_scored.columns[1:]
num_targets=len(targets)

In [14]:
train_features.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,0.549586,0.794990,-0.401905,-0.735057,-0.273013,-0.734081,...,0.391962,0.365030,0.660963,0.514674,0.171628,0.631482,0.055511,0.363927,0.484313,0.531946
1,id_000779bfc,trt_cp,72,D1,-0.142507,0.609193,0.126192,-0.021845,0.912762,0.561225,...,0.046709,0.600029,0.497948,0.271062,0.390401,0.530989,0.314315,0.312707,0.604819,0.753485
2,id_000a6266a,trt_cp,48,D1,0.245477,0.824916,1.337724,-0.165074,-0.114604,1.168270,...,-0.097893,-0.055826,0.565467,0.270739,-0.337358,0.049517,-0.059052,0.117182,-0.462421,0.722976
3,id_0015fd391,trt_cp,48,D1,-0.554596,-0.211053,-0.418993,0.468566,3.886483,-0.562946,...,-0.763498,-0.062650,-2.454912,-0.373754,-0.130313,-0.529347,-0.519612,-0.259094,0.077844,-0.323012
4,id_001626bd3,trt_cp,72,D2,-0.422582,-0.400341,0.773906,0.639331,1.302678,-0.575538,...,0.255353,0.244854,0.592911,0.752487,0.505696,0.058045,0.294186,0.406477,0.082609,0.736428


In [15]:
train_features=train_features[train_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test_features=test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

In [16]:
train_df=pd.concat([train_features,pd.get_dummies(train_features['cp_time'],prefix='cp_time')],axis=1)
train_df=pd.concat([train_df,pd.get_dummies(train_df['cp_type'],prefix='cp_type')],axis=1)
train_df=pd.concat([train_df,pd.get_dummies(train_df['cp_dose'],prefix='cp_dose')],axis=1)
train_df=train_df.drop(['cp_time','cp_type','cp_dose'],axis=1)

#test features,
test_df=pd.concat([test_features,pd.get_dummies(test_features['cp_time'],prefix='cp_time')],axis=1)
test_df=pd.concat([test_df,pd.get_dummies(test_df['cp_type'],prefix='cp_type')],axis=1)
test_df=pd.concat([test_df,pd.get_dummies(test_df['cp_dose'],prefix='cp_dose')],axis=1)
test_df=test_df.drop(['cp_time','cp_type','cp_dose'],axis=1)

In [17]:
train_df.drop('sig_id',axis=1,inplace=True)
test_df.drop('sig_id',axis=1,inplace=True)

In [18]:
num_features=train_df.shape[1]
num_features

878

In [19]:
#https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/166833
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [20]:
folds=MultilabelStratifiedKFold(n_splits=5,shuffle=False,random_state=42)
# train_df.loc[:,'folds']=-1
# for i,(trn_idx,val_idx) in enumerate(folds.split(train_df,train_targets_scored[targets])):
#     print(f'For fold {i} Train index {len(trn_idx)} Valid index {len(val_idx)}')
#     train_df.loc[val_idx,'folds']=i
    

In [21]:
class MOADataset():
    def __init__(self,data,label):
        self.data=data.values
        self.label=label.values
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,item):
        return {
            "x":torch.tensor(self.data[item,],dtype=torch.float),
            "y":torch.tensor(self.label[item,],dtype=torch.float)
        }
    
class MOADatasetTest():
    def __init__(self,data):
        self.data=data.values
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,item):
        return {
            "x":torch.tensor(self.data[item,],dtype=torch.float)
        }

In [22]:
class MOAnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1=nn.Linear(num_features,2048)
        self.bn1=nn.BatchNorm1d(2048)
        self.do1=nn.Dropout(0.3)
        self.l2=nn.Linear(2048,1024)
        self.bn2=nn.BatchNorm1d(1024)
        self.do2=nn.Dropout(0.2)
        self.l3=nn.Linear(1024,512)
        self.bn3=nn.BatchNorm1d(512)
        self.do3=nn.Dropout(0.4)
        self.out=nn.Linear(512,num_targets)
    
    def forward(self,x):
        x=self.bn1(self.l1(x))
        x=F.relu(self.do1(x))
        x=self.bn2(self.l2(x))
        x=F.relu(self.do2(x))
        x=self.bn3(self.l3(x))
        x=F.relu(self.do3(x))
        x=self.out(x)
        return x
        

In [23]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [24]:
# train_x,valid_x,train_tar,valid_tar=train_test_split(train_df,train_targets_scored.loc[:,targets],test_size=0.2,random_state=42)

In [25]:
# train_tar.head()

In [26]:
# train_x.head()

In [27]:
# train_dataset=MOADataset(data=train_x.values,
#                         label=train_tar.values)
# train_dataloader=DataLoader(train_dataset,batch_size=1024,shuffle=True,num_workers=4)
# valid_dataset=MOADataset(data=valid_x.values,
#                         label=valid_tar.values)
# valid_dataloader=DataLoader(valid_dataset,batch_size=1024,shuffle=True,num_workers=4)

In [28]:
#https://www.kaggle.com/ateplyuk/aptos-pytorch-starter-rnet50
#https://www.kaggle.com/chanhu/eye-inference-num-class-1-ver3
#https://www.kaggle.com/chanhu/eye-efficientnet-pytorch-lb-0-777
#https://www.kaggle.com/artgor/basic-eda-and-baseline-pytorch-model

In [29]:
model=MOAnn()
optimizer=optim.Adam(model.parameters(),lr=0.001)
scheduler=lr_scheduler.StepLR(optimizer,step_size=10)
criterion=nn.BCEWithLogitsLoss()
#criterion=LabelSmoothingLoss()
model=model.to(device)

In [30]:
def run_folds(fold,trn_x,trn_y,val_x,val_y,NUM_EPOCHS):
    print('-'*10)
    print(f'Running fold {fold+1}')
    best_loss=np.Inf
    train_dataset=MOADataset(data=trn_x,
                            label=trn_y)
    train_dataloader=DataLoader(train_dataset,batch_size=1024,shuffle=True)
    print(len(train_dataloader))
    valid_dataset=MOADataset(data=val_x,
                            label=val_y)
    valid_dataloader=DataLoader(valid_dataset,batch_size=1024,shuffle=True)
    test_dataset=MOADatasetTest(data=test_df)
    test_dataloader=DataLoader(test_dataset,batch_size=1024,shuffle=False,num_workers=4)
    test_preds=np.zeros((len(test_dataset),num_targets))
    print(len(valid_dataloader))
    for epoch in range(NUM_EPOCHS):
        counter=0
        avg_tr_loss=0.0
        model.train()
        for bi,d in enumerate(train_dataloader):
            inputs=d['x']
            labels=d['y']
            inputs=inputs.to(device,dtype=torch.float)
            labels=labels.to(device,dtype=torch.float)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                output=model(inputs)
                loss=criterion(output,labels)
                loss.backward()
                optimizer.step()
            avg_tr_loss+=loss.item() /len(train_dataloader)

        #print(f'Epoch {epoch+1} ')

        model.eval()
        avg_val_loss=0.0
        val_preds=[]
        for bi,d in enumerate(valid_dataloader):
            inputs=d['x']
            labels=d['y']
            inputs=inputs.to(device,dtype=torch.float)
            labels=labels.to(device,dtype=torch.float)
            with torch.no_grad():
                output=model(inputs)
            #print(type(val_output))
            loss=criterion(output,labels)
            avg_val_loss+=loss.item() /len(valid_dataloader)
            scheduler.step()
            val_output=torch.sigmoid(output).detach().cpu().squeeze().numpy()
            
            #print(f'Val Output {type(val_output)} and {val_output} and {val_output.shape}')
            val_preds.append(val_output)
            #print(f'Val preds {val_preds}')
            #print(f'Validation prediction shape after appending {len(val_preds)}')
        print(f'Epoch {epoch+1}  Training epoch Loss {avg_tr_loss} Validation epoch Loss {avg_val_loss}')
        val_preds=np.concatenate(val_preds)
        #print(f'Validation prediction shape after concatenating {val_preds.shape}')
        

        if avg_val_loss < best_loss:
            print(f'Validation loss {avg_val_loss} is less than best loss {best_loss}.Saving the model')
            torch.save(model.state_dict(),f'model_{fold}.bin')
            best_loss=avg_val_loss
        
    model.load_state_dict(torch.load(f'model_{fold}.bin'))
  
    model.eval()
    with torch.no_grad():
        for bi,d in enumerate(test_dataloader):
            inputs=d['x']
            inputs=inputs.to(device,dtype=torch.float)
            pred=model(inputs)
            #print(pred,pred.shape,torch.sigmoid(pred),pred.detach().cpu().sqeeze())
            test_preds[bi*1024:(bi+1)*1024]=torch.sigmoid(pred).detach().cpu().squeeze().numpy().ravel().reshape(-1,206)
            #print(type(test_preds))
    
    return val_preds,test_preds

In [31]:
oof=np.zeros((len(train_df),num_targets))
prediction=np.zeros((len(test_df),num_targets))
for i,(trn_idx,val_idx) in enumerate(folds.split(train_df,train_targets_scored[targets])):
    print(f'Starting fold {i+1}')
    trn_x,val_x=train_df.loc[trn_idx].reset_index(drop=True),train_df.loc[val_idx].reset_index(drop=True)
    trn_y,val_y=train_targets_scored.iloc[trn_idx,1:].reset_index(drop=True),train_targets_scored.iloc[val_idx,1:].reset_index(drop=True)
    print(trn_x.shape,val_x.shape,trn_y.shape,val_y.shape)
    op,test_predictions=run_folds(i,trn_x,trn_y,val_x,val_y,30)
    oof[val_idx]=op
    prediction+=test_predictions/5
#     print(f'Fold {i+1} completed')
#     print('_'*10)
    

Starting fold 1
(17558, 878) (4390, 878) (17558, 206) (4390, 206)
----------
Running fold 1
18
5
Epoch 1  Training epoch Loss 0.26939278489185703 Validation epoch Loss 0.11208584159612656
Validation loss 0.11208584159612656 is less than best loss inf.Saving the model
Epoch 2  Training epoch Loss 0.03273369972076681 Validation epoch Loss 0.024864024296402932
Validation loss 0.024864024296402932 is less than best loss 0.11208584159612656.Saving the model
Epoch 3  Training epoch Loss 0.02453139579544465 Validation epoch Loss 0.02371789924800396
Validation loss 0.02371789924800396 is less than best loss 0.024864024296402932.Saving the model
Epoch 4  Training epoch Loss 0.02420410524225897 Validation epoch Loss 0.02341998890042305
Validation loss 0.02341998890042305 is less than best loss 0.02371789924800396.Saving the model
Epoch 5  Training epoch Loss 0.023999758685628567 Validation epoch Loss 0.023426663130521774
Epoch 6  Training epoch Loss 0.024073860608041283 Validation epoch Loss 0.0

In [32]:
oof[:,1]

array([0.00608028, 0.0059279 , 0.00993256, ..., 0.00729524, 0.00609187,
       0.00770447])

In [33]:
np.array(train_targets_scored.iloc[:,1])

array([0, 0, 0, ..., 0, 0, 0])

In [34]:
#https://www.kaggle.com/yasufuminakama/moa-pytorch-nn-starter
score=0
for i in range(len(targets)):
    oof_loss=log_loss(train_targets_scored.iloc[:,i+1],oof[:,i])
    score+=oof_loss/206
print(f'{score}')

0.025417947493155627


In [35]:
prediction.shape

(3624, 206)

In [36]:
sample_submission.loc[~sample_submission['sig_id'].isin(te_ctrl_vehicle_idx),targets]

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
6,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3978,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3979,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3980,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [37]:
sample_submission.loc[~sample_submission['sig_id'].isin(te_ctrl_vehicle_idx),targets]=prediction
sample_submission.loc[sample_submission['sig_id'].isin(te_ctrl_vehicle_idx),targets]=0

In [38]:
sample_submission.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.009764,0.005168,0.006733,0.007064,0.008375,0.007573,0.008064,0.007684,0.008903,...,0.005892,0.006507,0.006718,0.006838,0.007763,0.007806,0.010858,0.006795,0.006107,0.007528
1,id_001897cda,0.007417,0.005142,0.006600,0.006842,0.006433,0.008626,0.007425,0.009621,0.010815,...,0.007851,0.004202,0.005006,0.009048,0.007143,0.008359,0.008146,0.005189,0.008678,0.007461
2,id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,id_00276f245,0.006155,0.005434,0.006718,0.007527,0.006836,0.009837,0.009298,0.011092,0.011460,...,0.007881,0.003762,0.004322,0.009751,0.007742,0.006761,0.008398,0.006838,0.007700,0.008002
4,id_0027f1083,0.008077,0.007194,0.007530,0.007798,0.008127,0.004913,0.008016,0.005793,0.004846,...,0.006619,0.004142,0.007421,0.006191,0.007983,0.004431,0.005072,0.004485,0.006682,0.006795


In [39]:
sample_submission.to_csv('submission.csv',index=False)